# Using a GAN to Generate Stress-strain Curves

## Problem Definition

Assuming bi-linear stress strain behavior of a material (characterized by $\sigma_y$, $E$, and $H$), generate sample stress-strain curves based on some initial samples of a stress-strain curve distrbution. 

<img src="bilinear.png" alt="Drawing" style="width: 300px;"/>

## Generating the training data

The training samples will be gathered by asuming independent, normal distributions for $\sigma_y$, $E$, and $H$.

  * $\sigma_y \sim \mathcal{N}(\mu=10, \sigma=0.5)$
  * $E \sim \mathcal{N}(\mu=1000, \sigma=50)$
  * $H \sim \mathcal{N}(\mu=50, \sigma=5)$

In [1]:
import numpy as np
%matplotlib inline
import matplotlib
import seaborn as sns
sns.set()
import matplotlib.pyplot as plt
import pandas as pd

Two helper functions

In [2]:
def get_stress(strains, E, s_y, H):
    e_y = s_y / E
    elastic_strains = strains.copy()
    elastic_strains[elastic_strains > e_y] = e_y
    plastic_strains = strains - elastic_strains
    stresses = elastic_strains*E + plastic_strains*H
    return stresses

In [3]:
def generate_samples(max_strain, n_strain, n_samples):
    strain = np.linspace(0, max_strain, n_strain + 1)[1:]
    stresses = np.empty((n_samples, n_strain))
    for i in range(n_samples):
        E = np.random.normal(1000, 50)
        s_y = np.random.normal(10, 0.5)
        H = np.random.normal(50, 5)
        stresses[i] = get_stress(strain, E, s_y, H)
    return stresses, strain

Make training data:

  * rows in stress_mat correspond to the stresses in a single stress strain curve (i.e. 1 sample)
  * columns in stress_mat correspond to a single strain value

In [10]:
N_SAMPLES = 1000
MAX_STRAIN = 0.02
NUM_STRAINS = 10

stress_mat, strains = generate_samples(MAX_STRAIN, NUM_STRAINS, N_SAMPLES)
print(stress_mat)
print(stress_mat.shape)
print(strains)
print(strains.shape)

[[ 2.03649733  4.07299466  6.10949198 ... 10.20641061 10.29498196
  10.3835533 ]
 [ 2.11073423  4.22146845  6.33220268 ... 10.67432581 10.77437103
  10.87441624]
 [ 1.92525831  3.85051662  5.77577492 ... 11.00379691 11.11289614
  11.22199537]
 ...
 [ 2.15279992  4.30559984  6.45839975 ... 10.90084816 11.0095516
  11.11825504]
 [ 2.04646009  4.09292017  6.13938026 ...  9.83165654  9.92636691
  10.02107729]
 [ 2.08663911  4.17327822  6.25991733 ...  9.79039347  9.87827202
   9.96615057]]
(1000, 10)
[0.002 0.004 0.006 0.008 0.01  0.012 0.014 0.016 0.018 0.02 ]
(10,)


Visualize the training distribution

In [11]:
from ipywidgets import interact
import ipywidgets as widgets

def plot_hist(x):
    fig, (ax_left, ax_right) = plt.subplots(1, 2, gridspec_kw={'width_ratios': [2, 1]},
                                       figsize=(16, 7))
    
    sns.violinplot(data=pd.DataFrame(data=stress_mat, columns=np.round(strains, 3)), 
                   ax=ax_left)
    ax_left.set(xlabel='strain', ylabel='stress')
    
    itemindex = np.argmin(abs(strains-x))
    sns.distplot(stress_mat[:, itemindex], bins=20)
    ax_right.set(xlim=(np.min(stress_mat), np.max(stress_mat)),
                 xlabel='stresses at strain of %.3f' % strains[itemindex])
    
interact(plot_hist, x=(0.0, 0.02, 0.002))

interactive(children=(FloatSlider(value=0.01, description='x', max=0.02, step=0.002), Output()), _dom_classes=…

<function __main__.plot_hist(x)>

## Train a GAN to produce samples that match this distribution